# Weights and Biases Example

This example shows a simple way to log metrics to Weights and Biases from Pachyderm. It uses a Pachyderm secret to establish a secure connection to your W&B server, and pushes the logs there as your Pachyderm job runs. 

<p align="center">
	<img src='images/wandb_screenshot.png' width='600' title=''>
</p>


In [1]:
!pachctl version

COMPONENT           VERSION             
pachctl             2.1.3               
pachd               2.1.3               


## Create Pachyderm Secret 
First, we will create a Pachyderm secret that contains our Weights and Biases API key. 

In [2]:
# Edit the secrets.json to contain your API key
!pachctl create secret -f secrets.json

## Upload dataset to Pachyderm
In this example, we use the MNIST dataset. We'll download it locally, create a Pachyderm repo and then push the MNIST dataset to the repo. 

In [3]:
!pachctl create repo mnist

In [ ]:
# Download the dataset locally
!python download_mnist.py --data-location ./data/

# Push the dataset to Pachyderm
!cd data && pachctl put file -r mnist@master:/ -f ./

## Create Training Pipeline
Our training pipeline consists of a `json` file that tells orchestrates how our python code will run. Here it takes the `mnist` data repository as input, runs the `mnist.py` training script, and logs the information to Weights and Biases. 

Our API key is passed via a secret to our pipeline, creating an environment variable `WANDB_API_KEY` when the pipeline is run. W&B uses this API key to establish a connection with our logging server. 

<p align="center">
	<img src='images/pachyderm_mnist_screenshot.png' width='600' title=''>
</p>

In [5]:
!cat train-mnist.json

{
  "pipeline": {
    "name": "train-mnist"
  },
  "description": "Train MNIST example logging to W&B.",
  "input": {
    "pfs": {
      "repo": "mnist",
      "branch": "master",
      "glob": "/"
    }
  },
  "transform": {
    "cmd": [
      "/bin/sh"
    ],
    "stdin": [
      "python mnist.py --data-location /pfs/mnist/ --wandb-project example-mnist-wandb"
    ],
    "image": "jimmywhitaker/wandb_mnist:0.1",
    "secrets": [
      {
        "name": "wandb",
        "env_var": "WANDB_API_KEY",
        "key": "key"
      }
    ]
  }
}

In [6]:
!pachctl create pipeline -f train-mnist.json

## Clean Up Example
Delete pipeline, dataset repo, and local files created. 

In [7]:
!pachctl delete pipeline train-mnist

!pachctl delete repo mnist

!rm -rf data/